<a href="https://colab.research.google.com/github/albertotrunk/depth2video/blob/main/02_inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade git+https://github.com/huggingface/diffusers.git transformers accelerate scipy

In [ ]:
import requests
from PIL import Image
from IPython.display import display
import numpy as np


url_mask = "https://raw.githubusercontent.com/albertotrunk/depth2video/main/example/depth.png?raw=true" #@param {type:"string"}
mask_image = Image.open(requests.get(url_mask, stream=True).raw)
img2_grayscale = mask_image.convert('L')
img2_a = np.array(img2_grayscale)

mask = np.array(img2_grayscale)

strength = .8  # this controls the "strength" of the prompt relative to the init image

d = int(255 * (1-strength))
mask *= 255-d # convert from range [0,1] to [0,255]
mask += d

mask_image = Image.fromarray(mask)


url_init = "https://raw.githubusercontent.com/albertotrunk/depth2video/main/example/color.png?raw=true" #@param {type:"string"}
init_image = Image.open(requests.get(url_init, stream=True).raw)
display(init_image)


display(mask_image)


prompt = "Scarlett Johansson" #@param {type:"string"}


In [ ]:
# fuzzing the mask edges generally improves synthesis results

from torchvision.transforms import GaussianBlur

blur = GaussianBlur(11,20)
mask_image = blur(mask_image)

In [ ]:
import torch
from diffusers import StableDiffusionInpaintPipeline
from IPython.display import display

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16,
).to("cuda")

#image and mask_image should be PIL images.
#The mask structure is white for inpainting and black for keeping as is
image = pipe(prompt=prompt, image=init_image, mask_image=mask_image).images[0]
display(image)

In [ ]:
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

images = [init_image,mask_image,image]


plt.figure(figsize=(20,10))
columns = 5
for i, image in enumerate(images):
    plt.subplot(len(images) / columns + 1, columns, i + 1)
    plt.imshow(image)